In [1]:
pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.0/277.0 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from ortools.linear_solver import pywraplp
import time


Let's start with product 1 and 7


In [ ]:
class data_generator_1_7():

    def __init__(self):
        self.periods = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

        self.product = [1, 7]  # Product IDs

        # ✅ Map each product to a family (1 = Cereal, 7 = Fruits)
        self.product_family = {1: "cereal", 7: "fruits"}

        self.demands = [
            [0, 95, 110, 96, 86, 124, 83, 108, 114, 121, 110, 124, 104, 86, 87],
            [110, 93, 0, 112, 84, 124, 98, 101, 83, 87, 105, 118, 115, 106, 78]
        ]

        self.prod_cost = 50
        self.setup_cost = 100
        self.holding_cost = 5

        self.safety_stock = [
            [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
            [20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20]
        ]

        self.first_inventory = [83, 23]
        self.production_time = [
            [3.1, 3.2, 0],  # Cereal: mixing & packing in min
            [3.1, 0, 3.3]  # Fruits
        ]
        self.capacity_per_machine = [4600, 2000, 3000]  # Mixing, packing cereal, packing fruits
        self.cleaning_time = [30, 30]
        self.min_available_per_month__per_machine = [18240, 12768, 12768]



In [ ]:


def optimal_production_with_capacity(data):
    # Create the MIP solver with the SCIP backend
    m = pywraplp.Solver.CreateSolver('SCIP')

    # Objective variable
    obj = m.NumVar(0, 1e6, 'obj')

    # Decision variables
    setup = []
    production = []
    inventory = []

    for product in range(len(data.demands)):
        setup_temp = []
        production_temp = []
        inventory_temp = []
        for period in range(len(data.periods)):
            suffix = f'_{product}_{period}'
            setup_temp.append(m.BoolVar('setup' + suffix))
            production_temp.append(m.NumVar(0, sum(data.demands[product]), 'production' + suffix))
            inventory_temp.append(m.NumVar(0, sum(data.demands[product]), 'inventory' + suffix))
        setup.append(setup_temp)
        production.append(production_temp)
        inventory.append(inventory_temp)

    # Constraints
    for product in range(len(data.demands)):
        for period in range(len(data.periods)):
            if period == 0:
                m.Add(data.first_inventory[product] + production[product][period] ==
                      data.demands[product][period] + inventory[product][period])
            else:
                m.Add(inventory[product][period - 1] + production[product][period] ==
                      data.demands[product][period] + inventory[product][period])

            m.Add(production[product][period] <=
                  (sum(data.demands[product][period:]) + sum(data.safety_stock[product][period:])) * setup[product][period])
            m.Add(inventory[product][period] >= data.safety_stock[product][period])

    # Capacity constraints by machine and family
    for k in range(3):  # 0 = Mixing, 1 = Packing cereal, 2 = Packing fruits
        for period in range(len(data.periods)):
            if k == 0:
                m.Add(
                    sum(data.production_time[product][k] * production[product][period] +
                        data.cleaning_time[product] * setup[product][period]
                        for product in range(len(data.demands)))
                    <= data.min_available_per_month__per_machine[k]
                )
            elif k == 1:
                m.Add(
                    sum(data.production_time[product][k] * production[product][period]
                        for product in range(len(data.demands))
                        if data.product_family[data.product[product]] == "cereal")
                    <= data.min_available_per_month__per_machine[k]
                )
            elif k == 2:
                m.Add(
                    sum(data.production_time[product][k] * production[product][period]
                        for product in range(len(data.demands))
                        if data.product_family[data.product[product]] == "fruits")
                    <= data.min_available_per_month__per_machine[k]

                )
        # ADDITIONAL: Capacity per machine constraint (based on capacity_per_machine)
        for k in range(3):  # 0 = Mixing, 1 = Packing Cereal, 2 = Packing Fruits
            for period in range(len(data.periods)):
                if k == 0:
                    # Mixing applies to all products
                    m.Add(
                        sum(production[product][period]
                            for product in range(len(data.demands)))
                        <= data.capacity_per_machine[k]
                    )
                elif k == 1:
                    # Packing Cereal: only products in cereal family
                    m.Add(
                        sum(production[product][period]
                            for product in range(len(data.demands))
                            if data.product_family[data.product[product]] == "cereal")
                        <= data.capacity_per_machine[k]
                    )
                elif k == 2:
                    # Packing Fruits: only products in fruits family
                    m.Add(
                        sum(production[product][period]
                            for product in range(len(data.demands))
                            if data.product_family[data.product[product]] == "fruits")
                        <= data.capacity_per_machine[k]
                    )


    # Objective function
    m.Add(obj >= sum(
        data.prod_cost * production[product][period] +
        data.setup_cost * setup[product][period] +
        data.holding_cost * inventory[product][period]
        for product in range(len(data.demands))
        for period in range(len(data.periods))
    ))

    m.Minimize(obj)

    # Solve the model
    status = m.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        PRODUCTION_PLAN = {}
        for product in range(len(data.demands)):
            plan = {}
            for t, period in enumerate(data.periods):
                plan[period] = {
                    'production': production[product][t].solution_value(),
                    'setup': setup[product][t].solution_value(),
                    'inventory': inventory[product][t].solution_value()
                }
            PRODUCTION_PLAN[product] = plan
        return PRODUCTION_PLAN, m.Objective().Value()
    else:
        print("The problem does not have an optimal solution.")
        return None, None

# Run the model
start_time = time.time()
data = data_generator_1_7()  # Your data class
PRODUCTION_PLAN, obj = optimal_production_with_capacity(data)
end_time = time.time()

# Show results
if PRODUCTION_PLAN:
    for product_idx, product in enumerate(data.product):
        print(f"\nProduction Plan for Product {product}:\n")
        display(pd.DataFrame.from_dict(PRODUCTION_PLAN[product_idx]).round(0))

    print('\nObjective Value:', round(obj))
    print(f"Computation Time: {end_time - start_time:.3f} seconds")



Production Plan for Product 1:



,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
production,0.0,22.0,110.0,96.0,86.0,124.0,83.0,108.0,114.0,121.0,110.0,124.0,104.0,86.0,87.0
setup,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
inventory,83.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0



Production Plan for Product 7:



,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
production,107.0,93.0,0.0,112.0,84.0,124.0,98.0,101.0,83.0,87.0,105.0,118.0,115.0,106.0,78.0
setup,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
inventory,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0



Objective Value: 144715
Computation Time: 0.040 seconds


Now for product 1,2,7,8

In [ ]:
class data_generator_1278():

    def __init__(self):
        self.periods = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

        self.product = [1, 2, 7, 8]  # Product IDs
        self.family = ["cereal","fruits"]

        # ✅ Map each product to a family (1 = Cereal, 7 = Fruits)
        self.product_family = {
    1: "cereal",
    2: "cereal",
    7: "fruits",
    8: "fruits"
}
        self.product_index_to_family = [self.product_family[pid] for pid in self.product]


        self.demands = [
            [0, 95, 110, 96, 86, 124, 83, 108, 114, 121, 110, 124, 104, 86, 87],
            [98,	96,	96,	98,	103,	104,	122,	101,	89,	108,	101,	109,	106,	108,	76],
            [110, 93, 0, 112, 84, 124, 98, 101, 83, 87, 105, 118, 115, 106, 78],
            [85,	92,	101,	110,	93,	96,	120,	109,	121,	87,	92,	85,	91,	93,	109]
        ]

        self.prod_cost = 50
        self.setup_cost = 100
        self.holding_cost = 5

        self.safety_stock = [
            [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
            [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
            [20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20],
            [20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20]
        ]

        self.first_inventory = [83, 31, 23, 91]

        self.production_time = [
            [3.1, 3.2, 0],  # Cereal: mixing & packing in min
            [3.1, 0, 3.3]  # Fruits
        ]
        self.capacity_per_machine = [4600, 2000, 3000]  # Mixing, packing cereal, packing fruits
        self.cleaning_time = [30, 20, 30, 20]
        self.min_available_per_month__per_machine = [18240, 12768, 12768]



In [ ]:


def optimal_production_with_capacity(data):
    # Create the MIP solver with the SCIP backend
    m = pywraplp.Solver.CreateSolver('SCIP')

    # Objective variable
    obj = m.NumVar(0, 1e6, 'obj')

    # Decision variables
    setup = []
    production = []
    inventory = []

    for product in range(len(data.demands)):
        setup_temp = []
        production_temp = []
        inventory_temp = []
        for period in range(len(data.periods)):
            suffix = f'_{product}_{period}'
            setup_temp.append(m.BoolVar('setup' + suffix))
            production_temp.append(m.NumVar(0, sum(data.demands[product]), 'production' + suffix))
            inventory_temp.append(m.NumVar(0, sum(data.demands[product]), 'inventory' + suffix))
        setup.append(setup_temp)
        production.append(production_temp)
        inventory.append(inventory_temp)

    # Constraints
    for product in range(len(data.demands)):
        for period in range(len(data.periods)):
            if period == 0:
                m.Add(data.first_inventory[product] + production[product][period] ==
                      data.demands[product][period] + inventory[product][period])
            else:
                m.Add(inventory[product][period - 1] + production[product][period] ==
                      data.demands[product][period] + inventory[product][period])

            m.Add(production[product][period] <=
                  (sum(data.demands[product][period:]) + sum(data.safety_stock[product][period:])) * setup[product][period])
            m.Add(inventory[product][period] >= data.safety_stock[product][period])

    # Capacity constraints by machine and family
    for k in range(3):  # 0 = Mixing, 1 = Packing cereal, 2 = Packing fruits
        for period in range(len(data.periods)):
            if k == 0:
                m.Add(
                    sum(data.production_time[family][k] * production[product][period] +
                        data.cleaning_time[product] * setup[product][period]
                        for family in range(len(data.family)))
                    <= data.min_available_per_month__per_machine[k]
                )
            elif k == 1:
                m.Add(
                    sum(data.production_time[family][k] * production[product][period]
                        for family in range(len(data.family))
                        for product in range(len(data.demands))
                        if data.product_family[data.product[product]] == "cereal")
                    <= data.min_available_per_month__per_machine[k]
                )
            elif k == 2:
                m.Add(
                    sum(data.production_time[family][k] * production[product][period]
                        for family in range(len(data.family))
                        for product in range(len(data.demands))
                        if data.product_family[data.product[product]] == "fruits")
                    <= data.min_available_per_month__per_machine[k]

                )
        # ADDITIONAL: Capacity per machine constraint (based on capacity_per_machine)
        for k in range(3):  # 0 = Mixing, 1 = Packing Cereal, 2 = Packing Fruits
            for period in range(len(data.periods)):
                if k == 0:
                    # Mixing applies to all products
                    m.Add(
                        sum(production[product][period]
                            for product in range(len(data.demands)))
                        <= data.capacity_per_machine[k]
                    )
                elif k == 1:
                    # Packing Cereal: only products in cereal family
                    m.Add(
                        sum(production[product][period]
                            for product in range(len(data.demands))
                            if data.product_family[data.product[product]] == "cereal")
                        <= data.capacity_per_machine[k]
                    )
                elif k == 2:
                    # Packing Fruits: only products in fruits family
                    m.Add(
                        sum(production[product][period]
                            for product in range(len(data.demands))
                            if data.product_family[data.product[product]] == "fruits")
                        <= data.capacity_per_machine[k]
                    )


    # Objective function
    m.Add(obj >= sum(
        data.prod_cost * production[product][period] +
        data.setup_cost * setup[product][period] +
        data.holding_cost * inventory[product][period]
        for product in range(len(data.demands))
        for period in range(len(data.periods))
    ))

    m.Minimize(obj)

    # Solve the model
    status = m.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        PRODUCTION_PLAN = {}
        for product in range(len(data.demands)):
            plan = {}
            for t, period in enumerate(data.periods):
                plan[period] = {
                    'production': production[product][t].solution_value(),
                    'setup': setup[product][t].solution_value(),
                    'inventory': inventory[product][t].solution_value()
                }
            PRODUCTION_PLAN[product] = plan
        return PRODUCTION_PLAN, m.Objective().Value()
    else:
        print("The problem does not have an optimal solution.")
        return None, None

# Run the model
start_time = time.time()
data = data_generator_1278()  # Your data class
PRODUCTION_PLAN, obj = optimal_production_with_capacity(data)
end_time = time.time()

# Show results
if PRODUCTION_PLAN:
    for product_idx, product in enumerate(data.product):
        print(f"\nProduction Plan for Product {product}:\n")
        display(pd.DataFrame.from_dict(PRODUCTION_PLAN[product_idx]).round(0))

    print('\nObjective Value:', round(obj))
    print(f"Computation Time: {end_time - start_time:.3f} seconds")



Production Plan for Product 1:



,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
production,0.0,22.0,110.0,96.0,86.0,124.0,83.0,108.0,114.0,121.0,110.0,124.0,104.0,86.0,87.0
setup,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
inventory,83.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0



Production Plan for Product 2:



,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
production,77.0,96.0,96.0,98.0,103.0,104.0,122.0,101.0,89.0,108.0,101.0,109.0,106.0,108.0,76.0
setup,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
inventory,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0



Production Plan for Product 7:



,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
production,107.0,93.0,0.0,112.0,84.0,124.0,98.0,101.0,83.0,87.0,105.0,118.0,115.0,106.0,78.0
setup,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
inventory,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0



Production Plan for Product 8:



,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
production,14.0,92.0,101.0,110.0,93.0,96.0,120.0,109.0,121.0,87.0,92.0,85.0,91.0,93.0,109.0
setup,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
inventory,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0



Objective Value: 295315
Computation Time: 0.040 seconds


Now for products, 1,2,3,4,7,8,9,10

In [ ]:
class data_generator_1278():

    def __init__(self):
        self.periods = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

        self.product = [1, 2, 3, 4, 7, 8, 9, 10]  # Product IDs
        self.family = ["cereal","fruits"]

        # ✅ Map each product to a family (1 = Cereal, 7 = Fruits)
        self.product_family = {
    1: "cereal",
    2: "cereal",
    3: "cereal",
    4: "cereal",
    7: "fruits",
    8: "fruits",
    9: "fruits",
    10: "fruits"
}
        self.product_index_to_family = [self.product_family[pid] for pid in self.product]


        self.demands = [
            [0, 95, 110, 96, 86, 124, 83, 108, 114, 121, 110, 124, 104, 86, 87],
            [98,	96,	96,	98,	103,	104,	122,	101,	89,	108,	101,	109,	106,	108,	76],
            [106,	0,	89,	123,	96,	105,	83,	82, 112,	109,	119,	85,	99,	80,	123],
            [98,	121,	0,	105,	98,	96,	101,	81,	117,	76,	103,	81,	95,	105, 102],
            [110, 93, 0, 112, 84, 124, 98, 101, 83, 87, 105, 118, 115, 106, 78],
            [85,	92,	101,	110,	93,	96,	120,	109,	121,	87,	92,	85,	91,	93,	109],
            [122,	116,	109,	0,	105,	108,	88,	98,	77,	90,	110,	102,	107,	99,	96],
            [120,	124,	94,	105,	92,	86,	101,	106,	75,	109,	83,	95,	79,	108,	100]
        ]

        self.prod_cost = 50
        self.setup_cost = 100
        self.holding_cost = 5

        self.safety_stock = [
            [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
            [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
            [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
            [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
            [20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20],
            [20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20],
            [20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20],
            [20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20]
        ]

        self.first_inventory = [83, 31, 11, 93, 23, 91, 83, 34]

        self.production_time = [
            [3.1, 3.2, 0],  # Cereal: mixing & packing in min
            [3.1, 0, 3.3]  # Fruits
        ]
        self.capacity_per_machine = [4600, 2000, 3000]  # Mixing, packing cereal, packing fruits
        self.cleaning_time = [30, 20, 30, 40, 30, 20, 10, 50]
        self.min_available_per_month__per_machine = [18240, 12768, 12768]



In [ ]:


def optimal_production_with_capacity(data):
    # Create the MIP solver with the SCIP backend
    m = pywraplp.Solver.CreateSolver('SCIP')

    # Objective variable
    obj = m.NumVar(0, 1e6, 'obj')

    # Decision variables
    setup = []
    production = []
    inventory = []

    for product in range(len(data.demands)):
        setup_temp = []
        production_temp = []
        inventory_temp = []
        for period in range(len(data.periods)):
            suffix = f'_{product}_{period}'
            setup_temp.append(m.BoolVar('setup' + suffix))
            production_temp.append(m.NumVar(0, sum(data.demands[product]), 'production' + suffix))
            inventory_temp.append(m.NumVar(0, sum(data.demands[product]), 'inventory' + suffix))
        setup.append(setup_temp)
        production.append(production_temp)
        inventory.append(inventory_temp)

    # Constraints
    for product in range(len(data.demands)):
        for period in range(len(data.periods)):
            if period == 0:
                m.Add(data.first_inventory[product] + production[product][period] ==
                      data.demands[product][period] + inventory[product][period])
            else:
                m.Add(inventory[product][period - 1] + production[product][period] ==
                      data.demands[product][period] + inventory[product][period])

            m.Add(production[product][period] <=
                  (sum(data.demands[product][period:]) + sum(data.safety_stock[product][period:])) * setup[product][period])
            m.Add(inventory[product][period] >= data.safety_stock[product][period])

    # Capacity constraints by machine and family
    for k in range(3):  # 0 = Mixing, 1 = Packing cereal, 2 = Packing fruits
        for period in range(len(data.periods)):
            if k == 0:
                m.Add(
                    sum(data.production_time[family][k] * production[product][period] +
                        data.cleaning_time[product] * setup[product][period]
                        for family in range(len(data.family)))
                    <= data.min_available_per_month__per_machine[k]
                )
            elif k == 1:
                m.Add(
                    sum(data.production_time[family][k] * production[product][period]
                        for family in range(len(data.family))
                        for product in range(len(data.demands))
                        if data.product_family[data.product[product]] == "cereal")
                    <= data.min_available_per_month__per_machine[k]
                )
            elif k == 2:
                m.Add(
                    sum(data.production_time[family][k] * production[product][period]
                        for family in range(len(data.family))
                        for product in range(len(data.demands))
                        if data.product_family[data.product[product]] == "fruits")
                    <= data.min_available_per_month__per_machine[k]

                )
        # ADDITIONAL: Capacity per machine constraint (based on capacity_per_machine)
        for k in range(3):  # 0 = Mixing, 1 = Packing Cereal, 2 = Packing Fruits
            for period in range(len(data.periods)):
                if k == 0:
                    # Mixing applies to all products
                    m.Add(
                        sum(production[product][period]
                            for product in range(len(data.demands)))
                        <= data.capacity_per_machine[k]
                    )
                elif k == 1:
                    # Packing Cereal: only products in cereal family
                    m.Add(
                        sum(production[product][period]
                            for product in range(len(data.demands))
                            if data.product_family[data.product[product]] == "cereal")
                        <= data.capacity_per_machine[k]
                    )
                elif k == 2:
                    # Packing Fruits: only products in fruits family
                    m.Add(
                        sum(production[product][period]
                            for product in range(len(data.demands))
                            if data.product_family[data.product[product]] == "fruits")
                        <= data.capacity_per_machine[k]
                    )


    # Objective function
    m.Add(obj >= sum(
        data.prod_cost * production[product][period] +
        data.setup_cost * setup[product][period] +
        data.holding_cost * inventory[product][period]
        for product in range(len(data.demands))
        for period in range(len(data.periods))
    ))

    m.Minimize(obj)

    # Solve the model
    status = m.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        PRODUCTION_PLAN = {}
        for product in range(len(data.demands)):
            plan = {}
            for t, period in enumerate(data.periods):
                plan[period] = {
                    'production': production[product][t].solution_value(),
                    'setup': setup[product][t].solution_value(),
                    'inventory': inventory[product][t].solution_value()
                }
            PRODUCTION_PLAN[product] = plan
        return PRODUCTION_PLAN, m.Objective().Value()
    else:
        print("The problem does not have an optimal solution.")
        return None, None

# Run the model
start_time = time.time()
data = data_generator_1278()  # Your data class
PRODUCTION_PLAN, obj = optimal_production_with_capacity(data)
end_time = time.time()

# Show results
if PRODUCTION_PLAN:
    for product_idx, product in enumerate(data.product):
        print(f"\nProduction Plan for Product {product}:\n")
        display(pd.DataFrame.from_dict(PRODUCTION_PLAN[product_idx]).round(0))

    print('\nObjective Value:', round(obj))
    print(f"Computation Time: {end_time - start_time:.3f} seconds")



Production Plan for Product 1:



,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
production,0.0,22.0,110.0,96.0,86.0,124.0,83.0,108.0,114.0,121.0,110.0,124.0,104.0,86.0,87.0
setup,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
inventory,83.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0



Production Plan for Product 2:



,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
production,77.0,96.0,96.0,98.0,103.0,104.0,122.0,101.0,89.0,108.0,101.0,109.0,106.0,108.0,76.0
setup,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
inventory,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0



Production Plan for Product 3:



...


Production Plan for Product 9:



,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
production,59.0,116.0,109.0,0.0,105.0,108.0,88.0,98.0,77.0,90.0,110.0,102.0,107.0,99.0,96.0
setup,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
inventory,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0



Production Plan for Product 10:



,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
production,106.0,124.0,94.0,105.0,92.0,86.0,101.0,106.0,75.0,109.0,83.0,95.0,79.0,108.0,100.0
setup,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
inventory,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0



Objective Value: 582165
Computation Time: 0.142 seconds


Now for all

In [32]:
class data_generator_1278():

    def __init__(self):
        self.periods = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

        self.product = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]  # Product IDs
        self.family = ["cereal","fruits"]

        # ✅ Map each product to a family (1 = Cereal, 7 = Fruits)
        self.product_family = {
    1: "cereal",
    2: "cereal",
    3: "cereal",
    4: "cereal",
    5: "cereal",
    6: "cereal",
    7: "fruits",
    8: "fruits",
    9: "fruits",
    10: "fruits",
    11: "fruits",
    12: "fruits"

}
        self.product_index_to_family = [self.product_family[pid] for pid in self.product]


        self.demands = [ #per week
            [0, 95, 110, 96, 86, 124, 83, 108, 114, 121, 110, 124, 104, 86, 87],
            [98,	96,	96,	98,	103,	104,	122,	101,	89,	108,	101,	109,	106,	108,	76],
            [106,	0,	89,	123,	96,	105,	83,	82, 112,	109,	119,	85,	99,	80,	123],
            [98,	121,	0,	105,	98,	96,	101,	81,	117,	76,	103,	81,	95,	105, 102],
            [0,	124,	113,	123,	123,	79,	111,	98,	97,	80,	98,	124,	78,	108,	109],
            [103,	102,	0, 	95, 107, 105,	107,	105,	75,	93,	115,	113,	111,	105,	85],
            [110, 93, 0, 112, 84, 124, 98, 101, 83, 87, 105, 118, 115, 106, 78],
            [85,	92,	101,	110,	93,	96,	120,	109,	121,	87,	92,	85,	91,	93,	109],
            [122,	116,	109,	0,	105,	108,	88,	98,	77,	90,	110,	102,	107,	99,	96],
            [120,	124,	94,	105,	92,	86,	101,	106,	75,	109,	83,	95,	79,	108,	100],
            [117,	96,	78,	0,	108,	87,	114,	107,	110,	94,	104,	101,	108,	110,	80],
            [125,	112,	75,	0,	116,	103,	122,	88,	85,	84,	76,	102,	84,	88,	82]
        ]

        self.prod_cost = 50
        self.setup_cost = 100
        self.holding_cost = 5

        self.safety_stock = [
            [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
            [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
            [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
            [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
            [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
            [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
            [20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20],
            [20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20],
            [20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20],
            [20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20],
            [20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20],
            [20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20]
        ]

        self.first_inventory = [83, 31, 11, 93, 82, 72, 23, 91, 83, 34, 61, 82]

        self.production_time = [
            [3, 2, 0],  # Cereal: mixing & packing in min
            [3, 0, 1]  # Fruits
        ]

        self.cleaning_time = [30, 20, 30, 40, 40, 10, 30, 20, 10, 50, 30, 20] #in minutes
        self.min_available_per_month__per_machine = [4560, 3192, 3192] #per week



In [33]:

def optimal_production_with_capacity(data):
    # Create the MIP solver with the SCIP backend
    m = pywraplp.Solver.CreateSolver('SCIP')

    # Objective variable
    obj = m.NumVar(0, 1e6, 'obj')

    # Decision variables
    setup = []
    production = []
    inventory = []

    for product in range(len(data.demands)):
        setup_temp = []
        production_temp = []
        inventory_temp = []
        for period in range(len(data.periods)):
            suffix = f'_{product}_{period}'
            setup_temp.append(m.BoolVar('setup' + suffix))
            production_temp.append(m.NumVar(0, sum(data.demands[product]), 'production' + suffix))
            inventory_temp.append(m.NumVar(0, sum(data.demands[product]), 'inventory' + suffix))
        setup.append(setup_temp)
        production.append(production_temp)
        inventory.append(inventory_temp)

    # Constraints
    for product in range(len(data.demands)):
        for period in range(len(data.periods)): #Demand satisfaction cosntraint
            if period == 0:
                m.Add(data.first_inventory[product] + production[product][period] ==
                      data.demands[product][period] + inventory[product][period])
            else:
                m.Add(inventory[product][period - 1] + production[product][period] ==
                      data.demands[product][period] + inventory[product][period])

#Setup constraint + big M
            m.Add(production[product][period] <=
                  (sum(data.demands[product][period:]) + sum(data.safety_stock[product][period:])) * setup[product][period])
            m.Add(inventory[product][period] >= data.safety_stock[product][period])

    # Shared Capacity constraints by machine and family
    for k in range(3):  # 0 = Mixing, 1 = Packing cereal, 2 = Packing fruits
      for period in range(len(data.periods)):
          if k == 0:
              # Mixing: all products use it
              m.Add(
                  sum(
                      data.production_time[data.family.index(data.product_family[data.product[product]])][k] * production[product][period] +
                      data.cleaning_time[product] * setup[product][period]
                      for product in range(len(data.demands))
                  ) <= data.min_available_per_month__per_machine[k]
              )
          elif k == 1:
              # Packing Cereal: only cereal products
              m.Add(
                  sum(
                      data.production_time[0][k] * production[product][period] +
                      data.cleaning_time[product] * setup[product][period]
                      for product in range(len(data.demands))
                      if data.product_family[data.product[product]] == "cereal"
                  ) <= data.min_available_per_month__per_machine[k]
              )
          elif k == 2:
              # Packing Fruits: only fruit products
              m.Add(
                  sum(
                      data.production_time[1][k] * production[product][period] +
                      data.cleaning_time[product] * setup[product][period]
                      for product in range(len(data.demands))
                      if data.product_family[data.product[product]] == "fruits"
                  ) <= data.min_available_per_month__per_machine[k]
              )


    # Objective function
    m.Add(obj >= sum(
        data.prod_cost * production[product][period] +
        data.setup_cost * setup[product][period] +
        data.holding_cost * inventory[product][period]
        for product in range(len(data.demands))
        for period in range(len(data.periods))
    ))

    m.Minimize(obj)

    # Solve the model
    status = m.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        PRODUCTION_PLAN = {}
        for product in range(len(data.demands)):
            plan = {}
            for t, period in enumerate(data.periods):
                plan[period] = {
                    'production': production[product][t].solution_value(),
                    'setup': setup[product][t].solution_value(),
                    'inventory': inventory[product][t].solution_value()
                }
            PRODUCTION_PLAN[product] = plan
        return PRODUCTION_PLAN, m.Objective().Value()
    else:
        print("The problem does not have an optimal solution.")
        return None, None

# Run the model
start_time = time.time()
data = data_generator_1278()  # Your data class
PRODUCTION_PLAN, obj = optimal_production_with_capacity(data)
end_time = time.time()

# Show results
if PRODUCTION_PLAN:
    for product_idx, product in enumerate(data.product):
        print(f"\nWeekly Production Plan for Product {product}:\n")
        display(pd.DataFrame.from_dict(PRODUCTION_PLAN[product_idx]).round(0))

    print('\nObjective Value:', round(obj))
    print(f"Computation Time: {end_time - start_time:.3f} seconds")



Weekly Production Plan for Product 1:



,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
production,0.0,22.0,110.0,96.0,86.0,124.0,83.0,108.0,114.0,121.0,110.0,124.0,104.0,86.0,87.0
setup,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
inventory,83.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0



Weekly Production Plan for Product 2:



,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
production,77.0,96.0,96.0,98.0,103.0,104.0,122.0,101.0,89.0,108.0,101.0,109.0,106.0,108.0,76.0
setup,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
inventory,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0



Weekly Production Plan for Product 3:



...


Weekly Production Plan for Product 11:



,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
production,76.0,96.0,78.0,0.0,108.0,87.0,114.0,107.0,110.0,94.0,104.0,101.0,108.0,110.0,80.0
setup,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
inventory,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0



Weekly Production Plan for Product 12:



,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
production,63.0,112.0,75.0,0.0,116.0,103.0,122.0,88.0,85.0,84.0,76.0,102.0,84.0,88.0,82.0
setup,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
inventory,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0



Objective Value: 862875
Computation Time: 0.138 seconds


In [17]:
# Convert to matrix using existing PRODUCTION_PLAN
production_matrix = pd.DataFrame(
    data = [[PRODUCTION_PLAN[product_idx][period]['production']
             for period in data.periods]
            for product_idx in range(len(data.product))],
    index = data.product,
    columns = data.periods
).round(0)

print("Setup Plan Matrix:")
print(production_matrix)


Setup Plan Matrix:
       1      2      3      4      5      6      7      8      9      10  \
1     0.0   22.0  110.0   96.0   86.0  124.0   83.0  108.0  114.0  121.0   
2    77.0   96.0   96.0   98.0  103.0  104.0  122.0  101.0   89.0  108.0   
3   105.0    0.0   89.0  123.0   96.0  105.0   83.0   82.0  112.0  109.0   
4    15.0  121.0    0.0  105.0   98.0   96.0  101.0   81.0  117.0   76.0   
5     0.0   52.0  113.0  123.0  123.0   79.0  111.0   98.0   97.0   80.0   
6    41.0  102.0   -0.0   95.0  107.0  105.0  107.0  105.0   75.0   93.0   
7   107.0   93.0    0.0  112.0   84.0  124.0   98.0  101.0   83.0   87.0   
8    14.0   92.0  101.0  110.0   93.0   96.0  120.0  109.0  121.0   87.0   
9    59.0  116.0  109.0    0.0  105.0  108.0   88.0   98.0   77.0   90.0   
10  106.0  124.0   94.0  105.0   92.0   86.0  101.0  106.0   75.0  109.0   
11   76.0   96.0   78.0    0.0  108.0   87.0  114.0  107.0  110.0   94.0   
12   63.0  112.0   75.0    0.0  116.0  103.0  122.0   88.0   85.0   8

In [31]:
# Initialize machine usage tracker: [period][machine] = minutes used
machine_usage = {period: [0, 0, 0] for period in data.periods}  # 0=Mixing, 1=Packing Cereal, 2=Packing Fruits

for product_idx, product_id in enumerate(data.product):
    family = data.product_family[product_id]
    family_idx = 0 if family == "cereal" else 1

    for t_idx, period in enumerate(data.periods):
        prod_qty = PRODUCTION_PLAN[product_idx][period]['production']
        is_setup = PRODUCTION_PLAN[product_idx][period]['setup']

        # Time spent on mixing
        machine_usage[period][0] += data.production_time[family_idx][0] * prod_qty

        # Time spent on packing cereal or fruit
        if family == "cereal":
            machine_usage[period][1] += data.production_time[family_idx][1] * prod_qty
        else:
            machine_usage[period][2] += data.production_time[family_idx][2] * prod_qty

         # Add cleaning time (on both machines used)
        if is_setup >= 0.5:  # setup = cleaning needed
            cleaning_time = data.cleaning_time[product_idx]
            machine_usage[period][0] += cleaning_time  # always mixing
            if family == "cereal":
                machine_usage[period][1] += cleaning_time
            else:
                machine_usage[period][2] += cleaning_time


# Convert to DataFrame for display
import pandas as pd

machine_usage_df = pd.DataFrame.from_dict(
    machine_usage, orient='index',
    columns=["Mixing", "Packing Cereal", "Packing Fruits"]
).round(0)

print("\nMachine usage per period (in minutes):")
display(machine_usage_df)



Machine usage per period (in minutes):


,Mixing,Packing Cereal,Packing Fruits
1,4283.0,1214.0,944.0
2,2931.0,664.0,710.0
3,1921.0,862.0,231.0
4,3878.0,1786.0,473.0
5,3802.0,866.0,896.0
6,4088.0,1894.0,489.0
7,4063.0,826.0,1003.0
8,3046.0,1274.0,445.0
9,3964.0,1312.0,727.0
10,3773.0,1314.0,674.0


In [11]:
# Calculate total cleaning time per period
cleaning_time_per_period = setup_matrix.multiply(data.cleaning_time, axis=0).sum(axis=0)

# Add it as a new row
setup_matrix.loc['Total Cleaning Time (min)'] = cleaning_time_per_period

# Display the updated matrix
print("Setup Plan Matrix with Cleaning Time:")
display(setup_matrix)


Setup Plan Matrix with Cleaning Time:


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
1,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
6,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
8,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
9,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
10,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [26]:
for product_idx, product_id in enumerate(data.product):
    setups = sum(PRODUCTION_PLAN[product_idx][t]['setup'] for t in data.periods)
    print(f"Product {product_id}: {int(setups)} setups")


Product 1: 14 setups
Product 2: 15 setups
Product 3: 14 setups
Product 4: 14 setups
Product 5: 14 setups
Product 6: 14 setups
Product 7: 14 setups
Product 8: 15 setups
Product 9: 14 setups
Product 10: 15 setups
Product 11: 14 setups
Product 12: 14 setups
